## DICOM to Image

This notebook demonstrates how to convert a Digital Imaging and Communications in Medicine (DICOM) file to one or more images in PNG or JPEG format.

This example uses:
- `pydicom` library for reading and parsing DICOM files.
- `cv2` (installed as `opencv-python`) for image processing, color space conversions, and saving as JPG/PNG.
- `numpy` for numerical operations and handling arrays, useful to manipulate image pixel data.

In [ ]:
# Imports
import pydicom
import numpy as np
import cv2
from os import path

In [ ]:
# DICOM source file (`.dcm` extension).
SRC_PATH = "./example/input/sample_single-frame.dcm"

# Output path to store each frame.
OUT_PATH = "./example/output/"

# Output image type (can be "png" or "jpg").
EXT = "jpg"

### Load DICOM and ensure it contains image data

Not all DICOM files necessarily contain image data, because it is a standard format for both images and other medical data as well such as patient information, medical reports, etc. With this code fragment we ensure to only process DICOMs that contain image data.

In [ ]:
ds = pydicom.dcmread(SRC_PATH)

if "PixelData" not in ds:
    raise ValueError("DICOM file doesn't contain image information.")

### Convert source image type to RGB color space

The DICOM file may contain an image that is grayscale, RGB, YBR, or palette color. The code below converts the source image from its respective color space to RGB color space, which is a requirement for creating a PNG or JPG file.

Note that the "PALETTE COLOR" type is not handled, because it is more complicated and out of the scope of this example notebook.

In [ ]:
pixel_array = ds.pixel_array

# Handle each source image type.
if ds.PhotometricInterpretation == "MONOCHROME1":
    pixel_array = np.amax(pixel_array) - pixel_array
elif ds.PhotometricInterpretation == "MONOCHROME2":
    pass  # No need to convert because pixel values are already dark-to-bright in ascending order.
elif ds.PhotometricInterpretation == "YBR_FULL":
    pixel_array = cv2.cvtColor(pixel_array, cv2.COLOR_YCrCb2BGR)
elif ds.PhotometricInterpretation == "YBR_FULL_422":
    pixel_array = cv2.cvtColor(
        cv2.resize(pixel_array, (pixel_array.shape[1] * 2, pixel_array.shape[0])),
        cv2.COLOR_YCrCb2BGR,
    )
elif ds.PhotometricInterpretation == "RGB":
    pixel_array = cv2.cvtColor(pixel_array, cv2.COLOR_RGB2BGR)
elif ds.PhotometricInterpretation == "PALETTE COLOR":
    raise NotImplementedError("Palette color is not supported.")

# Normalize the pixel array to 0-255 for JPG/PNG.
pixel_array = cv2.normalize(pixel_array, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

### Save DICOM frames to the output folder

In [ ]:
n = 0

if hasattr(ds, "NumberOfFrames") and ds.NumberOfFrames > 1:
    for idx, frame in enumerate(pixel_array):
        cv2.imwrite(path.join(OUT_PATH, f"frame_{idx}.{EXT}"), frame)
        n += 1
else:
    cv2.imwrite(path.join(OUT_PATH, f"frame.{EXT}"), pixel_array)
    n += 1

print(f"Total {n} frame(s) saved to {OUT_PATH}")